Week 07 Project 06

In [15]:
#packages
from datetime import datetime
import sys
# sys.path.append('C:/Users/lesli/Documents/Duke/Masters/FinTech545')
# import fin_package as fin
import numpy as np
import math
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.integrate import quad

- Problem 1
- Current Stock Price $151.03
- Strike Price $165
- Current Date 03/13/2022
- Options Expiration Date 04/15/2022
- Risk Free Rate of 4.25%
- Continuously Compounding Coupon of 0.53%

Implement the closed form greeks for GBSM. Implement a finite difference derivative calculation. Compare the values between the two methods for both a call and a put.
Implement the binomial tree valuation for American options with and without discrete dividends. Assume the stock above:
- Pays dividend on 4/11/2022 of $0.88
Calculate the value of the call and the put. Calculate the Greeks of each. What is the sensitivity of the put and call to a change in the dividend amount?

In [17]:
def options_price(S, X, T, sigma, r, b, option_type='call'):
    """
    S: Underlying Price
    X: Strike
    T: Time to Maturity(in years)
    sigma: implied volatility
    r: risk free rate
    b: cost of carry -> r if black scholes, r-q if merton
    """
    d1 = (math.log(S/X) + (b + 0.5*sigma**2)*T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        return S * math.exp((b - r) * T) * norm.cdf(d1) - X * math.exp(-r * T) * norm.cdf(d2)
    else:
        return X * math.exp(-r * T) * norm.cdf(-d2) - S * math.exp((b - r) * T) * norm.cdf(-d1)
    

# function to calculate the option price error
def option_price_error(sigma, S, X, T, r, b, option_type, market_price):
    option_price = options_price(S, X, T, sigma, r, b, option_type)
    return abs(option_price - market_price)



def calculate_implied_volatility(curr_stock_price, strike_price, current_date, options_expiration_date, risk_free_rate, continuously_compounding_coupon, option_type, tol=1e-4, max_iter=300):
    S = curr_stock_price
    X = strike_price
    T = (options_expiration_date - current_date).days / 365
    r = risk_free_rate
    q = continuously_compounding_coupon
    b = r-q
    def calc_option_price(sigma):
        option_price = options_price(S, X, T, sigma, r, b, option_type)
        return option_price
    
    iteration = 0
    lower_vol = 0.001
    upper_vol = 15.0

    while iteration <= max_iter:
        mid_vol = (lower_vol + upper_vol) / 2
        option_price = calc_option_price(mid_vol)

        if abs(option_price) < tol:
            return mid_vol
        
        if option_price >0:
            upper_vol = mid_vol
        else:
            lower_vol = mid_vol

        iteration +=1

    raise ValueError( "Implied volatility calculation did not converge")


def greeks(underlying_price, strike_price, risk_free_rate, implied_volatility, continuous_dividend_rate, current_date, expiration_date, option_type):
    T = (expiration_date - current_date).days / 365
    r = risk_free_rate
    q = continuous_dividend_rate
    b = r - q
    S = underlying_price
    X = strike_price
    sigma = implied_volatility

    d1 = (np.log(S/X)+(b+(0.5*sigma**2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == 'call':
        delta = np.exp((b-r)*T)*norm.cdf(d1)
        theta = -1*(S*np.exp((b-r)*T)*norm.pdf(d1)*sigma)/(0.5*np.sqrt(T)) - (b-r)*S*np.exp((b-r)*T)*norm.cdf(d1) - r*X*np.exp(-r*T)*norm.cdf(d2)
        rho = T*X*np.exp(-r*T)*norm.cdf(d2)
        carry_rho = T*S*np.exp((b-r)*T)*norm.cdf(d2)

    else:
        delta = np.exp((b-r)*T)*(norm.cdf(d1) - 1)
        theta = -1*(S*np.exp((b-r)*T)*norm.pdf(d1)*sigma)/(0.5*np.sqrt(T)) + (b-r)*S*np.exp((b-r)*T)*norm.cdf(-1*d1) + r*X*np.exp(-r*T)*norm.cdf(-1*d2)
        rho = -1*T*X*np.exp(-r*T)*norm.cdf(-d2)
        carry_rho = -1*T*S*np.exp((b-r)*T)*norm.cdf(-d2)

    gamma = norm.pdf(d1)*np.exp((b-r)*T)/(S*sigma*(np.sqrt(T)))
    vega = S*np.exp((b-r)*T)*norm.pdf(d1)*np.sqrt(T)

    return delta, gamma, vega, theta, rho, carry_rho

# finite difference derivative calculation greeks
def greeks_df(underlying_price, strike_price, risk_free_rate, implied_volatility, continuous_dividend_rate, current_date, expiration_date, option_type, epsilon = 0.01):
    T = (expiration_date - current_date).days / 365
    r = risk_free_rate
    q = continuous_dividend_rate
    b = r - q
    S = underlying_price
    X = strike_price
    sigma = implied_volatility

    #options_price(S, X, T, sigma, r, b, option_type='call')

    def derivative(variable = None):
        if variable == "underlying": # delta
            up_price = options_price(S+epsilon, X, T, sigma, r, b, option_type)
            down_price = options_price(S-epsilon, X, T, sigma, r, b, option_type)
            return (up_price-down_price)/(2*epsilon)
        if variable == "double_underlying": # gamma
            up_price = options_price(S+epsilon, X, T, sigma, r, b, option_type)
            down_price = options_price(S-epsilon, X, T, sigma, r, b, option_type)
            reg_price = options_price(S, X, T, sigma, r, b, option_type)
            return (up_price+down_price-2*reg_price)/(epsilon**2)
        if variable == "implied_volatility": # vega
            up_price = options_price(S, X, T, sigma+epsilon, r, b, option_type)
            down_price = options_price(S, X, T, sigma-epsilon, r, b, option_type)
            return (up_price-down_price)/(2*epsilon)
        if variable == "time_to_maturity": # theta
            up_price = options_price(S, X, T+epsilon, sigma, r, b, option_type)
            down_price = options_price(S, X, T-epsilon, sigma, r, b, option_type)
            return -(up_price-down_price)/ (2*epsilon)
        if variable == 'risk_free_rate': #rho
            up_price = options_price(S, X, T, sigma, r+epsilon, b, option_type)
            down_price = options_price(S, X, T, sigma, r-epsilon, b, option_type)
            return (up_price-down_price)/(2*epsilon)
        if variable == 'cost_of_carry': # carry rho
            up_price = options_price(S, X, T, sigma, r, b+epsilon, option_type)
            down_price = options_price(S, X, T, sigma, r, b-epsilon, option_type)
            return (up_price-down_price)/(2*epsilon)

    delta = derivative("underlying")
    gamma = derivative("double_underlying")
    vega = derivative("implied_volatility")
    theta = derivative("time_to_maturity")
    rho = derivative("risk_free_rate")
    carry_rho = derivative("cost_of_carry")

    return delta, gamma, vega, theta, rho, carry_rho



def greeks_with_dividends(underlying_price, strike_price, risk_free_rate, implied_volatility, continuous_dividend_rate, current_date, expiration_date, option_type, div_dates, div_amounts):
    T = (expiration_date - current_date).days / 365
    r = risk_free_rate
    q = continuous_dividend_rate
    b = r - q
    S = underlying_price
    X = strike_price
    sigma = implied_volatility

    # Calculate present value of dividends
    pv_dividends = 0
    for div_date, div_amount in zip(div_dates, div_amounts):
        if div_date > current_date and div_date < expiration_date:
            pv_dividends += div_amount * np.exp(-r * (div_date - current_date).days / 365)

    # Adjust underlying price for dividends
    S_adj = S - pv_dividends

    d1 = (np.log(S_adj / X) + (b + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        delta = np.exp((b - r) * T) * norm.cdf(d1)
        theta = -1 * (S_adj * np.exp((b - r) * T) * norm.pdf(d1) * sigma) / (0.5 * np.sqrt(T)) - (b - r) * S_adj * np.exp((b - r) * T) * norm.cdf(d1) - r * X * np.exp(-r * T) * norm.cdf(d2)
        rho = T * X * np.exp(-r * T) * norm.cdf(d2)
        carry_rho = T * S_adj * np.exp((b - r) * T) * norm.cdf(d2)

    else:
        delta = np.exp((b - r) * T) * (norm.cdf(d1) - 1)
        theta = -1 * (S_adj * np.exp((b - r) * T) * norm.pdf(d1) * sigma) / (0.5 * np.sqrt(T)) + (b - r) * S_adj * np.exp((b - r) * T) * norm.cdf(-1 * d1) + r * X * np.exp(-r * T) * norm.cdf(-1 * d2)
        rho = -1 * T * X * np.exp(-r * T) * norm.cdf(-d2)
        carry_rho = -1 * T * S_adj * np.exp((b - r) * T) * norm.cdf(-d2)

    gamma = norm.pdf(d1) * np.exp((b - r) * T) / (S_adj * sigma * (np.sqrt(T)))
    vega = S_adj * np.exp((b - r) * T) * norm.pdf(d1) * np.sqrt(T)

    return delta, gamma, vega, theta, rho, carry_rho

def binomial_tree_option_pricing_american(underlying_price, strike_price, ttm, risk_free_rate, b, implied_volatility, num_steps, option_type):
    S = underlying_price
    X = strike_price
    T = ttm
    r = risk_free_rate
    sigma = implied_volatility
    N = num_steps

    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(b * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = np.exp(-r * dt)
    if option_type == 'call':
        z = 1
    else:
        z = -1
    
    def nNodeFunc(n):
        return int((n + 1) * (n + 2) / 2)

    def idxFunc(i, j):
        return nNodeFunc(j - 1) + i
    
    nNodes = nNodeFunc(N) - 1

    optionValues = np.empty(nNodes+1)  # Increase the size by 1

    for j in range(N, -1, -1):
        for i in range(j, -1, -1):
            idx = idxFunc(i, j)
            price = S * (u ** i) * (d ** (j - i))
            optionValues[idx] = max(0, z * (price - X))

            if j < N:
                optionValues[idx] = max(
                    optionValues[idx],
                    df * (pu * optionValues[idxFunc(i + 1, j + 1)] + pd * optionValues[idxFunc(i, j + 1)]),
                )

    return optionValues[0]

def binomial_tree_option_pricing_american_complete(underlying_price, strike_price, ttm, risk_free_rate, implied_volatility, num_steps, option_type, div_amounts = None, div_times = None):
    S = underlying_price
    X = strike_price
    T = ttm
    r = risk_free_rate
    sigma = implied_volatility
    N = num_steps

    if (div_amounts is None) or (div_times is None) or len(div_amounts) == 0 or len(div_times) == 0 or div_times[0] > N:
        return binomial_tree_option_pricing_american(S, X, T, risk_free_rate, risk_free_rate, implied_volatility, num_steps, option_type)
    
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    pu = (np.exp(r * dt) - d) / (u - d)
    pd = 1.0 - pu
    df = np.exp(-r * dt)
    if option_type == 'call':
        z = 1
    else:
        z = -1

    def nNodeFunc(n):
        return int((n + 1) * (n + 2) / 2)

    def idxFunc(i, j):
        return nNodeFunc(j - 1) + i + 1

    nDiv = len(div_times)
    n_nodes = nNodeFunc(N)
    option_values = np.empty(n_nodes + 1)  # Increase the size by 1

    for j in range(div_times[0], -1, -1):  # Use a float range for j
        for i in range(j, -1, -1):  # Use a float range for i
            idx = idxFunc(i, j)
            price = S * (u ** i) * (d ** (j - i))

            if j < div_times[0]:
                # times before or at the dividend working backward induction
                option_values[idx] = max(0, z * (price - X))
                option_values[idx] = max(option_values[idx], df * (pu * option_values[idxFunc(i + 1, j + 1)] + pd * option_values[idxFunc(i, j + 1)]))
            else:
                # time after the dividend
                val_no_exercise = binomial_tree_option_pricing_american_complete(price-div_amounts[0], X, ttm-div_times[0]*dt, risk_free_rate, implied_volatility, N-div_times[0], option_type, div_amounts[1:nDiv], div_times[1:nDiv] - div_times[0])
                val_exercise = max(0, z * (price - X))
                option_values[idx] = max(val_no_exercise, val_exercise)

    return option_values[0]


def return_calculate(prices_df, method="DISCRETE", date_column="Date"):
    vars = prices_df.columns
    n_vars = len(vars)
    vars = [var for var in vars if var != date_column]
    
    if n_vars == len(vars):
        raise ValueError(f"date_column: {date_column} not in DataFrame: {vars}")
    
    n_vars = n_vars - 1
    
    p = prices_df[vars].values
    n = p.shape[0]
    m = p.shape[1]
    p2 = np.empty((n-1, m))
    
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]
    
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\",\"DISCRETE\")")
    
    dates = prices_df[date_column].iloc[1:]
    
    # Create a new DataFrame with all columns
    data = {date_column: dates}
    for i in range(n_vars):
        data[vars[i]] = p2[:, i]
    
    out = pd.DataFrame(data)
    
    return out

def mle_normal_distribution_one_input(X):
    # Define the likelihood function for the normal distribution
    def log_likelihood(params, X):
        mean, var = params
        n = len(X)
        # Adjust X to be centered around the mean
        adjusted_X = X - mean
        # Get squared variance
        var2 = var**2
        # Calculate log likelihood
        log_likeli = -(n/2) * np.log(var2 * 2 * np.pi) - np.sum(adjusted_X**2) / (2 * var2)

        return -log_likeli

    # Calculate initial guess for mean and standard deviation
    mean_guess = np.mean(X, axis=0)
    std_dev_guess = np.std(X, axis=0)
    
    # Initial guess for optimization as a 1D array
    initial_params = np.array([mean_guess, std_dev_guess])

    # Perform optimization through minimization of log likelihood
    result = minimize(log_likelihood, initial_params, args=(X,))

    # Extract optimized parameters
    optimized_mean, optimized_std_dev = result.x

    # Print optimized mean and standard deviation
    print("Optimized Mean:", optimized_mean)
    print("Optimized Standard Deviation:", optimized_std_dev)

    return optimized_mean, optimized_std_dev


def calc_var_normal(mean, std_dev, alpha=0.05):
    VaR = norm.ppf(alpha, loc=mean, scale=std_dev)

    return -VaR

# Calculate ES for Normal
def calc_expected_shortfall_normal(mean, std_dev, alpha=0.05):
    
    # Calculate ES using the formula
    es = -1*mean + (std_dev * norm.pdf(norm.ppf(alpha, mean, std_dev)) / alpha)

    return es

# Calculate ES for Generalized T
def calc_expected_shortfall_t(mean, std_dev, df, alpha=0.05):
    # VaR for t dist
    var = -1*calc_var_t_dist(mean, std_dev, df, alpha=alpha)

    # PDF fucntion for t dist
    def t_pdf(x):
        return t.pdf(x, df, loc=mean, scale=std_dev)

    # Integrand for es
    def integrand(x):
        return x*t_pdf(x)

    # Calc ES using integration
    es, _ = quad(integrand, float("-inf"), var)

    return es/alpha


def calculate_portfolio_value_american(underlying_value, portfolio, current_date, dividend_payment_date, risk_free_rate):
    portfolio_value = 0.0

    for _, asset in portfolio.iterrows():
        if asset['Type'] == 'Option':
            S = underlying_value
            X = asset['Strike']
            expiration_date = datetime.strptime(asset['ExpirationDate'], "%m/%d/%Y")
            T = (expiration_date - current_date).days / 365.0
            option_type = asset['OptionType']
            dividend_payment_time = np.array([(dividend_payment_date - current_date).days])
            dividend_payment = np.array([1])

            implied_volatility = calculate_implied_volatility_newton(S, X, current_date, expiration_date, risk_free_rate, 0, option_type)

            # Calculate the american option price using tree method
            option_value = binomial_tree_option_pricing_american_complete(S, X, T, risk_free_rate, implied_volatility, (dividend_payment_date - current_date).days+1, option_type, dividend_payment, dividend_payment_time)

            # Add or subtract option value to the portfolio based on Holding (1 or -1)
            portfolio_value += asset['Holding'] * option_value
        elif asset['Type'] == 'Stock':
            # If it's a stock, just add its current price to the portfolio value
            portfolio_value += asset['Holding'] * (asset['CurrentPrice'] - underlying_value)

    return portfolio_value

In [7]:



# Implement closed form greeks for GBSM
## In fin_package
        
# Implement finite difference derivative calculation
## In fin_package

# Compare the values between the two methods for call and put
curr_stock_price = 151.03
strike_price = 165
current_date = datetime(2022, 3, 13)
options_expiration_date = datetime(2022, 4, 15)
risk_free_rate = 0.0425
continuously_compounding_coupon = 0.0053


option_type = "call"
implied_volatility = calculate_implied_volatility(curr_stock_price, strike_price, current_date, options_expiration_date, risk_free_rate, continuously_compounding_coupon, option_type)
function_greeks = greeks(curr_stock_price, strike_price, risk_free_rate, implied_volatility, continuously_compounding_coupon, current_date, options_expiration_date, option_type)
df_greeks = greeks_df(curr_stock_price, strike_price, risk_free_rate, implied_volatility, continuously_compounding_coupon, current_date, options_expiration_date, option_type, epsilon = 0.01)

print("Call Greeks ------------------------------------------------")
print(f"Function Delta: {function_greeks[0]}")
print(f"Discrete Delta: {df_greeks[0]}")

print(f"Function Gamma: {function_greeks[1]}")
print(f"Discrete Gamma: {df_greeks[1]}")

print(f"Function Vega: {function_greeks[2]}")
print(f"Discrete Vega: {df_greeks[2]}")

print(f"Function Theta: {function_greeks[3]}")
print(f"Discrete Theta: {df_greeks[3]}")

print(f"Function Rho: {function_greeks[4]}")
print(f"Discrete Rho: {df_greeks[4]}")

print(f"Function Carry Rho: {function_greeks[5]}")
print(f"Discrete Carry Rho: {df_greeks[5]}")

option_type = "put"
implied_volatility = calculate_implied_volatility(curr_stock_price, strike_price, current_date, options_expiration_date, risk_free_rate, continuously_compounding_coupon, 'call')
function_greeks = greeks(curr_stock_price, strike_price, risk_free_rate, implied_volatility, continuously_compounding_coupon, current_date, options_expiration_date, option_type)
df_greeks = greeks_df(curr_stock_price, strike_price, risk_free_rate, implied_volatility, continuously_compounding_coupon, current_date, options_expiration_date, option_type, epsilon = 0.01)

print("Put Greeks ------------------------------------------------")
print(f"Function Delta: {function_greeks[0]}")
print(f"Discrete Delta: {df_greeks[0]}")

print(f"Function Gamma: {function_greeks[1]}")
print(f"Discrete Gamma: {df_greeks[1]}")

print(f"Function Vega: {function_greeks[2]}")
print(f"Discrete Vega: {df_greeks[2]}")

print(f"Function Theta: {function_greeks[3]}")
print(f"Discrete Theta: {df_greeks[3]}")

print(f"Function Rho: {function_greeks[4]}")
print(f"Discrete Rho: {df_greeks[4]}")

print(f"Function Carry Rho: {function_greeks[5]}")
print(f"Discrete Carry Rho: {df_greeks[5]}")

# Implement binomial tree valuation for American options with and without discrete dividends
# Assume pays dividend on 4/11/2022 of $0.88

# separate implied volatility function to help puts converge
def calculate_implied_volatility_newton(curr_stock_price, strike_price, current_date, options_expiration_date, risk_free_rate, continuously_compounding_coupon, option_type, tol=1e-5, max_iter=500):
    S = curr_stock_price
    X = strike_price
    T = (options_expiration_date - current_date).days / 365
    r = risk_free_rate
    q = continuously_compounding_coupon
    b = r - q

    def calc_option_price(sigma):
        option_price = options_price(S, X, T, sigma, r, b, option_type)
        return option_price

    def calc_vega(sigma):
        d1 = (math.log(S / X) + (b + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
        vega = S * math.exp((b - r) * T) * norm.pdf(d1) * math.sqrt(T)
        return vega

    iteration = 0
    volatility = 0.2  # Initial guess

    while iteration <= max_iter:
        option_price = calc_option_price(volatility)
        vega = calc_vega(volatility)

        if abs(option_price) < tol:
            return volatility

        volatility = volatility - option_price / vega

        iteration += 1

    raise ValueError("Implied volatility calculation did not converge")


curr_stock_price = 151.03
strike_price = 165
current_date = datetime(2022, 3, 13)
options_expiration_date = datetime(2022, 4, 15)
ttm = (options_expiration_date-current_date).days / 365
risk_free_rate = 0.0425
continuously_compounding_coupon = 0.0053
dividend_payout_date = datetime(2022, 4, 11)
num_days_dividend = (dividend_payout_date - current_date).days
num_steps = num_days_dividend+1
div_times = np.array([num_days_dividend])
div_amounts = np.array([0.88])

# Calculate value of call and put for each
option_type = 'call'
implied_volatility = calculate_implied_volatility_newton(curr_stock_price, strike_price, current_date, options_expiration_date, risk_free_rate, continuously_compounding_coupon, option_type)

print("Call Binomial Trees ------------------------------------")
no_div_bt = binomial_tree_option_pricing_american_complete(curr_stock_price, strike_price, ttm, risk_free_rate, implied_volatility, num_steps, option_type)
print(f"No Dividend Binomial Tree: {no_div_bt}")
div_bt = binomial_tree_option_pricing_american_complete(curr_stock_price, strike_price, ttm, risk_free_rate, implied_volatility, num_steps, option_type, div_amounts, div_times)
print(f"Dividend Binomial Tree: {div_bt}")

option_type = 'put'
implied_volatility = calculate_implied_volatility_newton(curr_stock_price, strike_price, current_date, options_expiration_date, risk_free_rate, continuously_compounding_coupon, option_type, max_iter = 20000)

print("Put Binomial Trees ------------------------------------")
no_div_bt = binomial_tree_option_pricing_american_complete(curr_stock_price, strike_price, ttm, risk_free_rate, implied_volatility, num_steps, option_type)
print(f"No Dividend Binomial Tree: {no_div_bt}")
div_bt = binomial_tree_option_pricing_american_complete(curr_stock_price, strike_price, ttm, risk_free_rate, implied_volatility, num_steps, option_type, div_amounts, div_times)
print(f"Dividend Binomial Tree: {div_bt}")

# Calculate the Greeks of each
div_times = [datetime(2022, 4, 12)]
print("Call Binomial Trees ------------------------------------")
option_type = 'call'
no_div_greeks = greeks(curr_stock_price, strike_price, risk_free_rate, implied_volatility, continuously_compounding_coupon, current_date, options_expiration_date, option_type)
div_greeks = greeks_with_dividends(curr_stock_price, strike_price, risk_free_rate, implied_volatility, 0, current_date, options_expiration_date, option_type, div_times, div_amounts)
print(f"No Discrete Dividend Delta: {no_div_greeks[0]}")
print(f"Discrete Dividend Delta: {div_greeks[0]}")

print(f"No Discrete Dividend Gamma: {no_div_greeks[1]}")
print(f"Discrete Dividend Gamma: {div_greeks[1]}")

print(f"No Discrete Dividend Vega: {no_div_greeks[2]}")
print(f"Discrete Dividend Vega: {div_greeks[2]}")

print(f"No Discrete Dividend Theta: {no_div_greeks[3]}")
print(f"Discrete Dividend Theta: {div_greeks[3]}")

print(f"No Discrete Dividend Rho: {no_div_greeks[4]}")
print(f"Discrete DividendRho: {div_greeks[4]}")

print(f"No Discrete Dividend Carry Rho: {no_div_greeks[5]}")
print(f"Discrete Dividend Carry Rho: {div_greeks[5]}")

print("Put Binomial Trees ----------------------------------------")
option_type = 'put'
no_div_greeks = greeks(curr_stock_price, strike_price, risk_free_rate, implied_volatility, continuously_compounding_coupon, current_date, options_expiration_date, option_type)
div_greeks = greeks_with_dividends(curr_stock_price, strike_price, risk_free_rate, implied_volatility, 0, current_date, options_expiration_date, option_type, div_times, div_amounts)
print(f"No Discrete Dividend Delta: {no_div_greeks[0]}")
print(f"Discrete Dividend Delta: {div_greeks[0]}")

print(f"No Discrete Dividend Gamma: {no_div_greeks[1]}")
print(f"Discrete Dividend Gamma: {div_greeks[1]}")

print(f"No Discrete Dividend Vega: {no_div_greeks[2]}")
print(f"Discrete Dividend Vega: {div_greeks[2]}")

print(f"No Discrete Dividend Theta: {no_div_greeks[3]}")
print(f"Discrete Dividend Theta: {div_greeks[3]}")

print(f"No Discrete Dividend Rho: {no_div_greeks[4]}")
print(f"Discrete DividendRho: {div_greeks[4]}")

print(f"No Discrete Dividend Carry Rho: {no_div_greeks[5]}")
print(f"Discrete Dividend Carry Rho: {div_greeks[5]}")

Call Greeks ------------------------------------------------
Function Delta: 1.064211045659124e-06
Discrete Delta: 1.0642675100794655e-06
Function Gamma: 1.9411776879709066e-06
Discrete Gamma: 1.9412245644495965e-06
Function Vega: 0.00023855309623545742
Discrete Vega: 0.0008723816583899495
Function Theta: -0.0003204158333980998
Discrete Theta: -0.00010464095989855577
Function Rho: 1.4480859300575613e-05
Discrete Rho: -5.0694704835894944e-08
Function Carry Rho: 1.329946801436884e-05
Discrete Carry Rho: 1.4677310932645728e-05
Put Greeks ------------------------------------------------
Function Delta: -0.9995198724942445
Discrete Delta: -0.9995198724368493
Function Gamma: 1.9411776879709066e-06
Discrete Gamma: 1.9417711882852018e-06
Function Vega: 0.00023855309623545742
Discrete Vega: 0.0008723816577571597
Function Theta: 6.185210465533484
Discrete Theta: 6.18542645032818
Function Rho: -14.860582557513306
Discrete Rho: -1.2123716286737363
Function Carry Rho: -13.64821232609308
Discrete Ca

- Problem 2
Using the options portfolios from Problem3 last week (named problem2.csv in this week’s repo) and assuming :
- American Options
- Current Date 03/03/2023
- Current AAPL price is 165
- Risk Free Rate of 4.25%
- Dividend Payment of $1.00 on 3/15/2023

Using DailyPrices.csv. Fit a Normal distribution to AAPL returns – assume 0 mean return. Simulate AAPL returns 10 days ahead and apply those returns to the current AAPL price (above). 

Calculate Mean, VaR and ES.

Calculate VaR and ES using Delta-Normal.

Present all VaR and ES values a $ loss, not percentages. Compare these results to last week’s results.


In [18]:
# American options
current_date = datetime(2023, 3, 3)
dividend_payment_date = datetime(2023, 3, 15)
curr_aapl_price = 165
risk_free_rate = 0.0425

options_portfolios = pd.read_csv("problem2.csv")
# Portfolio Type Underlying Holding OptionType ExpirationDate Strike CurrentPrice
portfolios = options_portfolios['Portfolio'].unique()

daily_prices = pd.read_csv("DailyPrices.csv")
aapl = daily_prices[['Date', 'AAPL']]

# Calculate the log returns of AAPL
log_returns = return_calculate(aapl, method="LOG")

# Demean the series so there is 0 mean
aapl_mean = log_returns["AAPL"].mean()
centered_returns = log_returns["AAPL"] - aapl_mean

norm_mean, norm_std = mle_normal_distribution_one_input(centered_returns)

# Simulate AAPL returns 10 days ahead
def fit_ar1(data):
    n = len(data)
    x_t = data[:-1]
    x_t1 = data[1:]
    alpha = np.cov(x_t, x_t1)[0, 1] / np.var(x_t)
    epsilon = x_t1 - alpha * x_t
    sigma = np.std(epsilon)
    return alpha, sigma

# Created new AR(1) model due to issues with prior models
alpha, sigma = fit_ar1(centered_returns)

# predictions for the next 10 returns
num_steps = 10
predictions = np.empty(num_steps)
current_data = centered_returns.values

for tt in range(num_steps):
    epsilon_t = np.random.normal(0, sigma)
    next_return = alpha * current_data[-1] + epsilon_t
    predictions[tt] = next_return
    current_data = np.append(current_data, next_return)

# Apply those returns to AAPL price
# calculate prices from returns
def calculate_prices(initial_price, returns):
    prices = [initial_price]
    for r in returns:
        price_t = prices[-1] * (1 + r)
        prices.append(price_t)
    return prices[1:]

# calculate the next 10 prices
next_10_prices = calculate_prices(curr_aapl_price, predictions)

# Create a DataFrame for the simulated prices and dates
simulated_prices_df = pd.DataFrame({
    'Date': pd.bdate_range(start=current_date, periods=11),  # Including the current date
    'AAPL': [curr_aapl_price] + next_10_prices  # Including the current price
})

# Format the 'Date' column
simulated_prices_df['Date'] = simulated_prices_df['Date'].dt.strftime('%m/%d/%Y')

# Locate the index corresponding to '03/15/2023'
index_add = simulated_prices_df[simulated_prices_df['Date'] == '03/15/2023'].index[0]

# Add 1 to the all columns after for the dividend payment
# Date AAPL
simulated_prices_df.loc[index_add:, 'AAPL'] += 1
returns_aapl = simulated_prices_df['AAPL'].pct_change().dropna()
# Concatenate the existing daily_prices DataFrame with simulated_prices_df
combined_prices_df = pd.concat([aapl, simulated_prices_df[['Date', 'AAPL']]])

combined_prices_df['Date'] = pd.to_datetime(combined_prices_df['Date'])
combined_prices_df = combined_prices_df.sort_values(by='Date').reset_index(drop=True)

DNVaR = calc_var_normal(norm_mean, norm_std)*curr_aapl_price
DNES = calc_expected_shortfall_normal(norm_mean, norm_std, alpha = 0.05)*curr_aapl_price
print(f"Delta Normal VaR: ${DNVaR}")
print(f"Delta Normal ES: ${DNES}")


# Simulate returns for AAPL using Monte Carlo simulations
num_simulations = 10000
simulated_returns_aapl = np.random.choice(returns_aapl, size=(num_simulations, len(returns_aapl)), replace=True)

# Rest of the code remains the same
for portfolio in portfolios:
    portfolio_data = options_portfolios[options_portfolios['Portfolio'] == portfolio]
    portfolio_value = calculate_portfolio_value_american(curr_aapl_price, portfolio_data, current_date, dividend_payment_date, risk_free_rate)

    simulated_returns_aapl = np.random.choice(returns_aapl, size=(num_simulations, len(returns_aapl)), replace=True)[0, :]

    simulated_portfolio_values = portfolio_value * np.cumprod(1 + simulated_returns_aapl, axis=0)

    simulated_portfolio_returns = np.diff(simulated_portfolio_values) / (simulated_portfolio_values[:-1]+1e-8)

    # Calculate VaR and ES for the portfolio
    portfolio_var = np.percentile(simulated_portfolio_returns, 5) * portfolio_value
    portfolio_es = np.mean(simulated_portfolio_returns[simulated_portfolio_returns <= np.percentile(simulated_portfolio_returns, 5)]) * portfolio_value
    mean_portfolio_return = np.mean(simulated_portfolio_returns) * portfolio_value

    print(f"Portfolio: {portfolio}")
    print(f"Portfolio VaR: ${portfolio_var}")
    print(f"Portfolio ES: ${portfolio_es}")
    print(f"Mean Portfolio Return Value: ${mean_portfolio_return:.5f}")
    print("------------------------")

Optimized Mean: -3.357529308342207e-19
Optimized Standard Deviation: 0.022380778943343265
Delta Normal VaR: $6.074162394127957
Delta Normal ES: $29.444550205049023
Portfolio: Straddle
Portfolio VaR: $1.0846100104147202e+76
Portfolio ES: $1.084610010414722e+76
Mean Portfolio Return Value: $2745128905915684048485303283320139026367189607155919594133409440393223733248.00000
------------------------
Portfolio: SynLong
Portfolio VaR: $0.0
Portfolio ES: $0.0
Mean Portfolio Return Value: $0.00000
------------------------
Portfolio: CallSpread
Portfolio VaR: $0.0
Portfolio ES: $0.0
Mean Portfolio Return Value: $0.00000
------------------------
Portfolio: PutSpread
Portfolio VaR: $0.0
Portfolio ES: $0.0
Mean Portfolio Return Value: $0.00000
------------------------
Portfolio: Stock
Portfolio VaR: $0.15278683561165077
Portfolio ES: $0.26037068311801487
Mean Portfolio Return Value: $-0.15930
------------------------
Portfolio: Call 
Portfolio VaR: $4.980321962782256e+75
Portfolio ES: $5.4230500520